# Des

In all of the attacks below, we assume that the adversary has a number
of pairs $(x, y)$ where $y = DES_k(x)$, and $k$ is the secret key. (We will not
need the fact that the adversary can adaptively choose the values $x$ for which
it will receive $y = DES_k(x)$). Since DES is a Feistel network, the internal
f-function works on half a block at a time. We denote the left half of the
input by $x_1$ and the right half by $x_2$. Likewise, we denote the left half of the
output by $y_1$ and the right half of the output by $y_2$.

## one round

$\begin{cases}
x \to x_1||x_2\; ,  \; y\to y_1 || y _2 \\
y_1=x_2 \\
y_2=x_1\oplus f_1(x_2) \\
f_1(x)=S(E(x)\oplus k)
\end{cases}
$

we know $x$ and $y$, we want to know $k$.